In [1]:
from random import randint
import numpy as np
from functools import reduce
import unittest

In [2]:
class TooManyParams(Exception):
    def __init__(self):
        Exception.__init__(self, 'Too many parameters were given. You must specify just 1: size or new_matrix')
        
class NotEnoughParams(Exception):
    def __init__(self):
        Exception.__init__(self, "0 parameters were given. You must specify just 1: size or new_matrix")
        
class DimensionsAreNotEqual(Exception):
    def __init__(self):
        Exception.__init__(self, "Dimensions of the matrices are not equal.")

In [3]:
class MatrixService:
    
    @staticmethod
    def create_matrix(size):
        matrix = [[randint(0, 50) for i in range(size)] for i in range(size)]
        return matrix
    
    @staticmethod
    def np_check(matrix_a, matrix_b):

        size = len(matrix_a)
        matrix_a = np.array(matrix_a)
        matrix_b = np.array(matrix_b)

        if matrix_a.shape == matrix_b.shape:
            result = matrix_a.dot(matrix_b)
            return result

        else:
            raise DimensionsAreNotEqual
 
        
    @staticmethod
    def multiply_ab_matrices(matrix_a, matrix_b):
        
        size = len(matrix_a)
        a = np.array(matrix_a)
        b = np.array(matrix_b)
        if a.shape == b.shape:
            result = [[reduce(lambda x,y: x+y, [matrix_a[k][j] * matrix_b[j][i] for j in range(size)]) for i in range(size)] for k in range(size)]
            return result
        else:
            raise DimensionsAreNotEqual

In [4]:
class FileService:
    
    @staticmethod
    def write_to_file(matrix, file_name):
        
        with open(file_name, 'w') as file:
            data = '\n'.join(['\t'.join([str(i) for i in line]) for line in matrix])
            file.write(data)
    
    @staticmethod
    def read_from_file(file_name):
        
        with open(file_name, 'r') as file:
            lines = file.readlines()
            lines = [l.strip().split('\t') for l in lines]
            matrix = [[int(i) for i in line] for line in lines]
        return matrix

In [5]:
class Matrix:
    
    def __new__(cls, *args, **kwargs):
        size = kwargs.get('size', None)
        new_matrix = kwargs.get('new_matrix', None)

        if not size and not isinstance(new_matrix, (np.ndarray, list)):
            raise NotEnoughParams
        elif size and isinstance(new_matrix, (np.ndarray, list)):
            raise TooManyParams
        else:
            __obj = object.__new__(cls)
            return __obj
    
    def __init__(self, size=None, new_matrix=None):
        self.__matrix = MatrixService.create_matrix(size) if size else new_matrix 
    
    @property
    def matrix(self):
        return self.__matrix    

    def __mul__(self, other):
        result = MatrixService.multiply_ab_matrices(self.__matrix, other.matrix)
        return result

In [6]:
a = Matrix(size=5)

In [7]:
a.matrix

[[14, 48, 47, 4, 16],
 [48, 37, 2, 35, 48],
 [33, 24, 8, 19, 7],
 [39, 4, 14, 40, 33],
 [9, 44, 30, 5, 31]]

In [8]:
b = Matrix(size=5)

In [9]:
b.matrix

[[17, 35, 45, 1, 45],
 [49, 31, 9, 2, 25],
 [43, 5, 49, 34, 10],
 [49, 25, 29, 34, 43],
 [2, 1, 8, 23, 15]]

In [10]:
FileService.write_to_file(a.matrix, 'matrix_a.txt')
FileService.write_to_file(b.matrix, 'matrix_b.txt')

In [11]:
a = Matrix(new_matrix=FileService.read_from_file('matrix_a.txt'))
b = Matrix(new_matrix=FileService.read_from_file('matrix_b.txt'))

In [12]:
a.matrix

[[14, 48, 47, 4, 16],
 [48, 37, 2, 35, 48],
 [33, 24, 8, 19, 7],
 [39, 4, 14, 40, 33],
 [9, 44, 30, 5, 31]]

In [13]:
b.matrix

[[17, 35, 45, 1, 45],
 [49, 31, 9, 2, 25],
 [43, 5, 49, 34, 10],
 [49, 25, 29, 34, 43],
 [2, 1, 8, 23, 15]]

In [14]:
result1 = Matrix(new_matrix=MatrixService.multiply_ab_matrices(a.matrix, b.matrix))

In [15]:
result1.matrix

[[4839, 2329, 3609, 2212, 2712],
 [4526, 3760, 3990, 2484, 5330],
 [3026, 2421, 2700, 1160, 3087],
 [3487, 2592, 3901, 2642, 4210],
 [3906, 1985, 2664, 2000, 2485]]

In [16]:
result2 = Matrix(new_matrix=a*b)

In [17]:
result2.matrix

[[4839, 2329, 3609, 2212, 2712],
 [4526, 3760, 3990, 2484, 5330],
 [3026, 2421, 2700, 1160, 3087],
 [3487, 2592, 3901, 2642, 4210],
 [3906, 1985, 2664, 2000, 2485]]

In [18]:
FileService.write_to_file(result1.matrix, 'matrix_result.txt')
result = Matrix(new_matrix=FileService.read_from_file('matrix_result.txt'))

In [19]:
result.matrix

[[4839, 2329, 3609, 2212, 2712],
 [4526, 3760, 3990, 2484, 5330],
 [3026, 2421, 2700, 1160, 3087],
 [3487, 2592, 3901, 2642, 4210],
 [3906, 1985, 2664, 2000, 2485]]

In [20]:
np_result = Matrix(new_matrix=MatrixService.np_check(a.matrix, b.matrix))

In [21]:
np_result.matrix

array([[2775, 2589, 3273, 2932, 1870],
       [1058, 1795, 1090, 2597, 1175],
       [3262, 1883, 3567, 2360, 1370],
       [3739, 3816, 4265, 4978, 2736],
       [4848, 3503, 5530, 5169, 2279]])

In [249]:
class TestMatrixesMultiplication(unittest.TestCase):
    def test_multiplication(self):
        
        a = Matrix(size=5)
        b = Matrix(size=5)
        
        result = Matrix(new_matrix=MatrixService.multiply_ab_matrices(a.matrix, b.matrix))
        np_result = Matrix(new_matrix=MatrixService.np_check(a.matrix, b.matrix))
        
        check = np_result.matrix == result.matrix
        self.assertTrue(check.all())
        
    def test_TooManyParams(self):
        with self.assertRaises(TooManyParams):
            m = Matrix(size=5, new_matrix=[1, 2, 3])
    
    def test_NotEnoughParams(self):
        with self.assertRaises(NotEnoughParams):
            m = Matrix()
    
    def test_DimensionsAreNotEqual(self):
        a = Matrix(size=5)
        b = Matrix(size=6)
        with self.assertRaises(DimensionsAreNotEqual):
            m = a * b

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK
